In [9]:
import tensorflow as tf
tf.enable_eager_execution()
import keras
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Activation
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, Callback, ModelCheckpoint
import numpy as np
import pandas as pd
from collections import Counter
import re, os, sys, csv, math, operator

In [2]:
architecture = '1024x4D-512x3D-256x3D-128x3D-64x2-32x1-1'
activation = 'relu'
dropouts = [0.8, 0.9, 0.7, 0.8]
SEED = 1234567
num_input = 86

In [3]:
def define_model(data, architecture, num_labels=1, activation='relu', dropouts=[]):

        assert '-' in architecture
        archs = architecture.strip().split('-')
        net = data
        pen_layer = net
        prev_layer = net
        prev_num_outputs = None
        prev_block_num_outputs = None
        prev_stub_output = net
        for i in range(len(archs)):
            arch = archs[i]
            if 'x' in arch:
                arch = arch.split('x')
                num_outputs = int(re.findall(r'\d+',arch[0])[0])
                layers = int(re.findall(r'\d+',arch[1])[0])
                j = 0
                aux_layers = re.findall(r'[A-Z]',arch[0])
                for l in range(layers):
                    if aux_layers and aux_layers[0] == 'B':
                        if len(aux_layers)>1 and aux_layers[1]=='A':
                            print('adding fully connected layers with %d outputs followed by batch_norm and act' % num_outputs)

                            net = Dense(num_outputs, 
                                        name='fc' + str(i) + '_' + str(j),
                                        activation=None)(net)
                            net = BatchNormalization(center=True, scale=True, name='fc_bn'+str(i)+'_'+str(j))(net)
                            if activation =='relu': net = Activation('relu')(net)
                        else:
                            print('adding fully connected layers with %d outputs followed by batch_norm' % num_outputs)
                            net = Dense(num_outputs,
                                        name='fc' + str(i) + '_' + str(j),
                                        activation=activation)(net)
                            net = BatchNormalization(center=True, scale=True,
                                             name='fc_bn' + str(i) + '_' + str(j))(net)

                    else:
                        print('adding fully connected layers with %d outputs' % num_outputs)

                        net = Dense(num_outputs,
                                    name='fc' + str(i) + '_' + str(j), 
                                    activation=None)(net)
                        if activation =='relu': net = Activation('relu')(net)

                    if 'R' in aux_layers:
                        if prev_num_outputs and prev_num_outputs==num_outputs:
                            print('adding residual, both sizes are same')

                            net = net+prev_layer
                        else:
                            print('adding residual with fc as the size are different')
                            net = net + Dense(num_outputs,
                                                name='fc' + str(i) + '_' +'dim_'+ str(j),
                                                activation=None)(prev_layer)
                    prev_num_outputs = num_outputs
                    j += 1
                    prev_layer = net
                aux_layers_sub = re.findall(r'[A-Z]', arch[1])
                if 'R' in aux_layers_sub:
                    if prev_block_num_outputs and prev_block_num_outputs == num_outputs:
                        print('adding residual to stub, both sizes are same')
                        net = net + prev_stub_output
                    else:
                        print('adding residual to stub with fc as the size are different')
                        net = net + Dense(num_outputs,
                                         name='fc' + str(i) + '_' + 'stub_dim_' + str(j),
                                         activation=None)(prev_stub_output)

                if 'D' in aux_layers_sub and (num_labels == 1) and len(dropouts) > i:
                    print('adding dropout', dropouts[i])
                    net = Dropout(1.-dropouts[i], seed=SEED)(net, training=False)
                prev_stub_output = net
                prev_block_num_outputs = num_outputs
                prev_layer = net

            else:
                if 'R' in arch:
                    act_fun = 'relu'
                    print('using ReLU at last layer') 
                elif 'T' in arch:
                    act_fun = 'tanh'
                    print('using TanH at last layer')    
                else:
                    act_fun = None
                pen_layer = net
                print('adding final layer with ' + str(num_labels) + ' output')
                net = Dense(num_labels, name='fc' + str(i),
                            activation=None)(net)
                if activation =='relu': net = Activation('relu')(net)

        return net

In [5]:
inputs = Input(shape=(num_input,), name='elemental_fractions')
outputs = define_model(inputs, architecture, dropouts=dropouts)
model = Model(inputs=inputs, outputs=outputs, name= 'ElemNet')
model.summary(print_fn=lambda x: print(x))

model.load_weights("model/model_elemnet_deltae_tf2.h5")

print("Loaded model from disk")

adam = optimizers.Adam(lr=0.0001)
model.compile(loss=tf.keras.losses.mean_absolute_error, optimizer=adam, metrics=['mean_absolute_error'])

adding fully connected layers with 1024 outputs
adding fully connected layers with 1024 outputs
adding fully connected layers with 1024 outputs
adding fully connected layers with 1024 outputs
adding dropout 0.8
adding fully connected layers with 512 outputs
adding fully connected layers with 512 outputs
adding fully connected layers with 512 outputs
adding dropout 0.9
adding fully connected layers with 256 outputs
adding fully connected layers with 256 outputs
adding fully connected layers with 256 outputs
adding dropout 0.7
adding fully connected layers with 128 outputs
adding fully connected layers with 128 outputs
adding fully connected layers with 128 outputs
adding dropout 0.8
adding fully connected layers with 64 outputs
adding fully connected layers with 64 outputs
adding fully connected layers with 32 outputs
adding final layer with 1 output
Model: "ElemNet"
_________________________________________________________________
Layer (type)                 Output Shape              

In [6]:
#Contains 86 elements (Without Noble elements as it does not forms compounds in normal condition)
elements = ['H','Li','Be', 'B', 'C', 'N', 'O', 'F', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl',
            'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe','Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge',
            'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd',
            'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd',
            'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er','Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 
            'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu' ]

test_path = '../data/holdout10/experimental_test_set.csv'
prop = 'delta_e'

In [7]:
pred = pd.read_csv(test_path) 
pred_elem = pred[elements]

new_x_pred = pred_elem.values
new_x_pred = np.asarray(new_x_pred, dtype=np.float)

y_pred = pred[[prop]]
new_y_pred = np.array(y_pred)
new_y_pred.shape = (len(new_y_pred),)

In [ ]:
results = model.evaluate(new_x_pred, new_y_pred, batch_size=32)